Step 4 - ANOVA

This code calculates the significance of a difference between the control strain and studied strains at a particular time and condition.
As a result it produces an excel file in which you can peek the significantly different variants compared to control (but it will be explicitly visualised in the next Step)
The results of the code will be used in the Step 5 script designated to calculate heatmaps of fold changes between studied strains and control.

In [6]:
from datetime import datetime
import glob
import pandas as pd
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

If you want to use the latest excel file generated by the 'Step 1' script, you don't need to change anything in the following cell.

However, you might want to go back to some previous files generated by you - in such case mark the first part of the script with triple quotes, unquote the second part, and enter the filename that interests you.

In the output of the following cell you can see which file is used for the analysis.

In [7]:
# Import the data

IMPORT_PATH = os.path.join(os.getcwd(), "output_data")

# Use the latest excel file as an input (default option)
available_results = glob.glob(os.path.join(IMPORT_PATH, '*growth_data.xlsx'))
latest_file = max(available_results)
data = pd.read_excel(latest_file)

'''
Or enter the exact filename generated in the 01_data_import.ipynb script as an input

data = pd.read_excel('XXX_growth_data.xlsx')
'''

print(latest_file)

\\wsl.localhost\Ubuntu\home\marysia\stress_resistance_msc\output_data\2025-05-13_growth_data.xlsx


In [12]:
OUTPUT_PATH = os.path.join(os.getcwd(), "output_data")

timepoints = data['time'].unique()
conditions = data['condition'].unique()

current_date = datetime.now().strftime("%Y-%m-%d")

for time in timepoints:
    one_time = data[data['time'] == time]
    for condition in conditions:
        one_variant = one_time[one_time['condition'] == condition]

        if one_variant['strain_name'].nunique() > 1:  # ANOVA requires >1 group
            model = ols('growth ~ C(strain_name)', data=one_variant).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)

            tukey = pairwise_tukeyhsd(
                endog=one_variant['growth'],
                groups=one_variant['strain_name'],
                alpha=0.05
            )

            tukey_df = pd.DataFrame(
                data=tukey.summary().data[1:],
                columns=tukey.summary().data[0]
            )
            
            tukey_df['condition'] = condition
            tukey_df['time'] = time

            output_filename = f"{current_date}_Tukey_{condition}_{time}.xlsx"
            output_path = os.path.join(OUTPUT_PATH, output_filename)

            tukey_df.to_excel(output_path, index=False)